In [47]:
import pandas as pd
import json 

data = pd.read_csv("NEW_data_collection/data/gpt_3.5_data_qa.csv")
# data = data[data['api_res'].notnull()]

# api_responses = [json.loads(api_res)['choices'][0]['message']['content'] for api_res in data['api_res']]
# data['questions'] = [try: json.dumps(json.loads(api_output.strip()[7:-3])) except: None for api_output in api_responses]

In [50]:
data.to_csv("NEW_data_collection/data/gpt_3.5_data_qa.csv")

In [39]:
def parse_output_for_questions(api_res):
    try:
        # Traverse output to find content of response
        response = json.loads(api_res)['choices'][0]['message']['content']
        # Remomove first 7 and last 3 (they are "```json" and "```" respectively), then standardize the jsons
        questions = json.dumps(json.loads(response.strip()[7:-3]))
    except:
        # If there is an error, return None
        questions = None
    return questions

data['questions'] = data['api_res'].apply(parse_output_for_questions)

In [45]:
# Get cost for the run using the number of tokens used
total_prompt_tokens = sum([json.loads(api_res)['usage']['prompt_tokens'] for api_res in data[data['api_res'].notnull()]['api_res']])
total_completion_tokens = sum([json.loads(api_res)['usage']['completion_tokens'] for api_res in data[data['api_res'].notnull()]['api_res']])

PROMPT_COST = 0.001                     # Price per 1000 prompt tokens for GPT-3.5-turbo-1106 as of 12/22/2023
COMPLETION_COST = 0.002                 # Price per 1000 completion tokens for GPT-3.5-turbo-1106 as of 12/22/2023

total_cost = PROMPT_COST * (total_prompt_tokens/1000) + COMPLETION_COST * (total_completion_tokens/1000)
print(f"\nTotal cost: ${total_cost:.4f}")


Total cost: $4.2549


In [46]:
not_null_api_responses = data[data['api_res'].notnull()]['api_res']
total_prompt_tokens = sum([json.loads(api_res)['usage']['prompt_tokens'] for api_res in not_null_api_responses])
total_completion_tokens = sum([json.loads(api_res)['usage']['completion_tokens'] for api_res in not_null_api_responses])

total_cost = PROMPT_COST * (total_prompt_tokens/1000) + COMPLETION_COST * (total_completion_tokens/1000)
print(f"\nTotal cost: ${total_cost:.4f}")


Total cost: $4.2549


In [40]:
data['questions']

0       [{"question": "How can I request books and art...
1       [{"question": "How can I get quick help with a...
2       [{"question": "How can faculty and staff updat...
3       [{"question": "How can I schedule a campus tou...
4       [{"question": "How can I apply to SUNY Brockpo...
                              ...                        
2681    [{"question": "What resources does SUNY Brockp...
2682    [{"question": "How can I request an escort if ...
2683    [{"question": "How can I report a sexual assau...
2684    [{"question": "How can I get involved in the S...
2685    [{"question": "How can I access the weekly inc...
Name: questions, Length: 2686, dtype: object

In [34]:
# ([json.loads(api_output.strip()[7:-3]) for api_output in api_responses])

for api_output in api_responses:
    try:
        json.loads(api_output.strip()[7:-3])
    except:
        print(api_output)

```json
[
    {
        "question": "What are the typical pre-requisites for MTH 122, College Algebra?",
        "answer": "For MTH 122, College Algebra, the typical pre-requisite is MTH 111. However, it is very rare that an entering CHM major would first need to complete MTH 111. This is because moving MTH 122 to a spring semester would be less ideal for the typical incoming CHM major to be able to complete upper division elective courses in the senior year, which we offer as part of the American Chemical Society certified chemistry degree."
    },
    {
        "question": "What upper division elective courses are offered for the American Chemical Society certified chemistry degree?",
        "answer": "We offer a range of upper division elective courses as part of the American Chemical Society certified chemistry degree. These courses are designed to provide students with a comprehensive and rigorous education in chemistry, preparing them for a successful career in the field."
    }

0       False
1       False
2       False
3       False
4       False
        ...  
2681    False
2682    False
2683    False
2684    False
2685    False
Name: api_res, Length: 2686, dtype: bool

,url,data,api_res
0,https://brockport.illiad.oclc.org/illiad/logon...,"Request books, articles and more!\nNew to Inte...","{""id"": ""chatcmpl-8YiunSnMOGnsQJQ6Un9qjp1GD0Xr9..."
1,https://brockport.libanswers.com,1. Talk to Someone Now\n24/7 Chat - Best for q...,"{""id"": ""chatcmpl-8YiunWCsgOjU2RV335pvATJyHyM5F..."
2,https://today.brockport.edu/live/news/3550-how...,How to Update your Faculty/Staff Profile Webpa...,"{""id"": ""chatcmpl-8YiunCDbzf4yOSFXW21ADzRNNUMlF..."
3,https://www2.brockport.edu/,Main Page Content\nBe ThereEvents Calendar\nWh...,"{""id"": ""chatcmpl-8YiuneWGdlgh8SHCfTPOS0kc2lqTM..."
4,https://www2.brockport.edu/about/,"About\nSUNY Brockport is a selective, national...","{""id"": ""chatcmpl-8YiunW6JI9USkBChNDbCeJFF1fAlK..."
...,...,...,...
2681,https://www2.brockport.edu/support/university-...,V. Violence Against Women Act (VAWA) Crimes\nV...,"{""id"": ""chatcmpl-8YjjFRqlqF9G1MTNnir9ttHuJKJyl..."
2682,https://www2.brockport.edu/support/university-...,"Although we enjoy a safe campus, no area is to...","{""id"": ""chatcmpl-8YjjB1P7zHC9L1Fd4vkVfoT4i673G..."
2683,https://www2.brockport.edu/support/university-...,Sexual Assault and the Law\nSUNY Brockport has...,"{""id"": ""chatcmpl-8YjjCogCO7AtqnLxjneQDbST380q4..."
2684,https://www2.brockport.edu/support/university-...,The Student Patrol program allows students wor...,"{""id"": ""chatcmpl-8YjjFg6NB3FAjD5gMIeMzdvn8R3xL..."


In [ ]:
import datasets

datasets.load_dataset("json", data_files="NEW_data_collection/data/raw_scraper_output.json")



In [3]:
import json

# load in json
with open("NEW_data_collection/data/raw_scraper_output.json") as f:
    data = json.load(f)

In [ ]:
data

In [ ]:
with open("NEW_data_collection/data/raw_scraper_output.json", "r") as f:
    data = f.read()

In [ ]:
import pandas as pd
import json
import re
import os

gpt35 = pd.read_csv("NEW_data_collection/data/gpt-3.5-data-qa.csv")
gpt4 = pd.read_csv("NEW_data_collection/data/gpt-4-data-qa.csv")

In [ ]:
idx = 15
print("GPT-3.5 URL: "+gpt35['url'][idx])
print("GPT-4 URL: "+gpt4['url'][idx])
print("Context:\n"+gpt35['data'][idx])
print("GPT-3.5:\n"+json.dumps(json.loads(gpt35['questions'][idx]), indent=4))
print("GPT-4:\n"+json.dumps(json.loads(gpt4['questions'][idx]), indent=4))